In [2]:
import pandas as pd
from tqdm import tqdm

In [33]:
path = "../datasets/pid.csv"
df = pd.read_csv(path)
flag_dict = {e.strip(): True for e in df['PID']}


In [115]:
path = "../datasets/OneDrive_1_5-10-2021/1. XN_CBC_Tổng phân tích tế bào máu/TB.2015_modify.csv"
with open(path, 'r', encoding='utf8') as f:
    lines = f.readlines()

    arr = []
    for line in tqdm(lines):
        line = line.strip()
        out = line.split(',')
        PID = out[1].strip()
        if PID is not None and flag_dict.get(PID):
            arr.append(out)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1962764/1962764 [00:03<00:00, 585511.76it/s]


In [116]:
len(arr)

121443

In [120]:
df = pd.DataFrame(arr, columns=['intime',
 'PID',
 'SID',
 'Age',
 'sex',
 'Address',
 'LocationID',
 'LocationName',
 'Testcode',
 'Testname',
 'Result'])

df = df.assign(Testcode=df.Testcode.str.strip())
pattern = {e: None for e in df.Testcode.unique()}
df

,intime,PID,SID,Age,sex,Address,LocationID,LocationName,Testcode,Testname,Result
0,2016-12-20 15:00:06.430,04000490,201216-7736,1990,F,Phường Yên Hoà Quận Cầu Giấy Hà Nội,PKBz,P. Khám bệnh,224HST,Lượng Huyết sắc tố,110
1,2016-12-20 15:00:06.430,04000490,201216-7736,1990,F,Phường Yên Hoà Quận Cầu Giấy Hà Nội,PKBz,P. Khám bệnh,225HCT,Hematocrit,0.371
2,2016-12-20 15:00:06.430,04000490,201216-7736,1990,F,Phường Yên Hoà Quận Cầu Giấy Hà Nội,PKBz,P. Khám bệnh,226MCV,MCV,68.0
3,2016-12-20 15:00:06.430,04000490,201216-7736,1990,F,Phường Yên Hoà Quận Cầu Giấy Hà Nội,PKBz,P. Khám bệnh,227MCH,MCH,20.2
4,2016-12-20 15:00:06.430,04000490,201216-7736,1990,F,Phường Yên Hoà Quận Cầu Giấy Hà Nội,PKBz,P. Khám bệnh,228MCHC,MCHC,297
...,...,...,...,...,...,...,...,...,...,...,...
121438,2016-10-27 10:09:56.000,15391343,271016-8507,2015,M,Xã Đông Kinh Huyện Đông Hưng Thái Bình,THALw,Điều trị Thalassemia,227MCH,MCH,22.1
121439,2016-10-27 10:09:56.000,15391343,271016-8507,2015,M,Xã Đông Kinh Huyện Đông Hưng Thái Bình,THALw,Điều trị Thalassemia,228MCHC,MCHC,312
121440,2016-10-27 10:09:56.000,15391343,271016-8507,2015,M,Xã Đông Kinh Huyện Đông Hưng Thái Bình,THALw,Điều trị Thalassemia,233RDW-CV,RDW-CV,25.1
121441,2016-10-27 10:09:56.000,15391343,271016-8507,2015,M,Xã Đông Kinh Huyện Đông Hưng Thái Bình,THALw,Điều trị Thalassemia,234SLTC,Số lượng tiểu cầu,358


In [152]:
m = 0
c = 0
dict_list = []
ex_df = None
for pid, sub_df in df.groupby('PID'):
    c += 1
    m = max(m, sub_df.shape[0])
    if m ==9:
        ex_df = sub_df
        break
print(m, c)

9 1


In [155]:
def handle_group(ex_df):
    obj = pattern.copy()
    df2 = ex_df[['intime', 'Testcode', 'Result']]
    df2 = df2.assign(indate=df2['intime'].str.split(' ').str[0])
    df2.apply(pd.to_numeric, errors='coerce', downcast='float')
    df2.dropna(inplace=True)
    m = 0
    selected_df = None
    indate = None
    for date, sub_df in df2.groupby('indate'):
        if m <= sub_df.shape[0]:
            m = sub_df.shape[0]
            indate = date
            selected_df = sub_df

    main_df = selected_df[['Testcode','Result']]
    main_dict = main_df.set_index('Testcode').to_dict()['Result']
    obj.update(main_dict)
    return obj

In [160]:
m = 0
c = 0
dict_list = []
for pid, sub_df in tqdm(df.groupby('PID')):
    c += 1
    m = max(m, sub_df.shape[0])
    tgt = handle_group(sub_df)
    tgt['PID'] = pid
    dict_list.append(tgt)
print(m, c)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1212/1212 [00:06<00:00, 187.23it/s]

1561 1212


In [162]:
pd.DataFrame(dict_list).set_index("PID")

,224HST,225HCT,226MCV,227MCH,228MCHC,233RDW-CV,234SLTC,246SLBC,223SLHC
PID,,,,,,,,,
04000490,110,0.371,68.0,20.2,297,13.9,182,6.09,5.45
04001108,118,0.377,69.0,21.6,313,16.3,181,4.69,5.46
04001112,129,0.422,61.6,18.8,306,15.9,157,8.62,6.85
04001155,115,0.373,63.4,19.5,308,15.5,299,4.68,5.89
04001167,108,0.347,63.6,19.9,312,16.5,295,9.35,5.46
...,...,...,...,...,...,...,...,...,...
15040069,131,0.406,85.4,27.5,322,17.0,221,4.75,4.75
15063400,103,0.322,79.6,25.4,319,None,162,8.31,4.05
15176273,110,0.351,57.6,18.0,313,14.6,273,12.14,6.09


In [165]:
str(['intime',
 'PID',
 'SID',
 'Age',
 'sex',
 'Address',
 'LocationID',
 'LocationName',
 'Testcode',
 'Testname',
 'Result'])

"['intime', 'PID', 'SID', 'Age', 'sex', 'Address', 'LocationID', 'LocationName', 'Testcode', 'Testname', 'Result']"

In [163]:
print("")

SyntaxError: EOL while scanning string literal (<ipython-input-163-2b7be194dc08>, line 1)